<a href="https://colab.research.google.com/github/WittmannF/course/blob/master/day-3/assignment-3-cats-dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3 - Cats vs Dogs Classifier
In this project we are going to use transfer learning in order to buid a classifier to distinguish cats and dogs. As database, we will be using a subset from the Kaggle Competition: https://www.kaggle.com/c/dogs-vs-cats/. While the original dataset has 25000 images, you are going to use only 2000 images for training the model. As you will notice by the end of this project, thanks to transfer learning, we can get  great classification results even with smaller datasets. 

> **NOTE:** *Make sure to complete all the lines of code with a `TODO:` comment.*

## 1. Access the Dataset
Run the cells bellow in order to mount your Google Drive (if using colab) and download the dataset. The dataset is also available to be manually downloaded in [this repository](https://github.com/dl7days/datasets).

### 1.1 Mount Google Drive (if using Colab)
Let's mount Google Drive in order to download the dataset. This way you won't have to always download again the dataset when accessing the notebook.

In [0]:
import sys, os

MOUNT_GDRIVE = True # Choose Google Drive
USING_COLAB = 'google.colab' in sys.modules

# Update here with the folder of the files of your course
COURSE_DIRECTORY = '/content/drive/My Drive/course/day-3'

if MOUNT_GDRIVE and USING_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Create course directory if it doesn't exist (if not cloned from github)
    if not os.path.exists(COURSE_DIRECTORY):
        os.makedirs(COURSE_DIRECTORY)
    # Open course directory
    os.chdir(COURSE_DIRECTORY)

## 1.2 Download Dataset
Now, let's download the dataset if it is the first time you are running this cell (i.e., if the dataset folder was not found): 

In [0]:
import os

DATA_PATH = 'cats-dogs-data/'
DATA_URL = 'https://github.com/dl7days/datasets/raw/master/cats-dogs-data.zip'
ZIP_FILENAME = 'cats-dogs-data.zip'

if not os.path.exists(DATA_PATH):# If dataset folder doesn't exist
    try: # Then try downloading and unzipping it
        print("Downloading Dataset...")
        os.system(f"wget {DATA_URL}")

        print("Unzipping Dataset")
        os.system(f"unzip {ZIP_FILENAME}")

        print("Removing .zip file")
        os.system(f"rm {ZIP_FILENAME}")
    except Exception as e: # If there's an error, ask to download manually
        print(f"Something went wrong. Please download the dataset manually at {DATA_URL}")
        print(f'The following exception was thrown:\n{e}')
else:
    print(f'Dataset folder {DATA_PATH} has been found')

In [0]:
# Assign training and validation folders
TRAIN_PATH = f'{DATA_PATH}train/'
VALID_PATH = f'{DATA_PATH}valid/'

## 2. Visualize Images from the Dataset
Now, let's visualize some of the images available in the training folder. First let's load a list with all filepaths:

In [0]:
import glob

filepaths = glob.glob(TRAIN_PATH+'/cat/*.jpg')
for f in glob.glob(TRAIN_PATH+'/dog/*.jpg'):
    filepaths.append(f)

Now, let's visualize a random image from the dataset. Every time you run the next cell, a random image (either of a dog or a cat) is going to be displayed:

In [0]:
from keras.preprocessing.image import load_img
import random

load_img(random.choice(filepaths))

## 3. Applying Transfer Learning

### 3.1 Defining Image Data Generators
Your first task is to define a training and a validation generator for loading the dataset. Follow those steps:
1. First of al, you will choose any model of your choice in https://keras.io/applications to be imported (for example ResNet50)
2. Next, import the model itself and the `preprocess_input` function (for example `from keras.applications.resnet50 import ResNet50, preprocess_input`)
3. Import the ImageDataGenerator from the [image preprocessing module](https://keras.io/preprocessing/image/)
4. Initialize the data generator class including the argument `preprocessing_function` as the `preprocess_input` function imported from the transfer learning model. This way all images are going to be preprocessed in the same settings of the images that were trained in the imported model.
5. Define the training and validationg generators using the method `flow_from_directory`. 
    - Add the variables `TRAIN_PATH` and `VALID_PATH` as the path of each of each one. 
    - Set the dimension (224, 224) as the argument `target_size` (`TARGET_SHAPE[:2]` also works).
    - As class mode, choose 'sparse'

In [0]:
# TODO: Import the model and the preprocess_input function


# TODO: Import the ImageDataGenerator class


# Shape in which all images are going to be reshaped
TARGET_SHAPE = (224, 224, 3)

# TODO: Initialize the data generator class 
datagen = ...

# TODO: Create the training and validation generators using the method flow_from_directory
train_gen = ...
valid_gen = ...

### 3.2 Defining Base Model
Now you are going to define import a model from http://keras.io/applications, and add a fully connected layer in the end. When initializing the transfer learning model, make sure to set `input_shape` as (224, 224, 3) (or TARGET_SHAPE) and set `include_top` as False:

In [0]:
# 1. TODO: Import any additional packages
from keras.models import Sequential
from keras.layers import ... # Import layers such as Dense, Flatten
from keras.applications. ... import ... # Import your choice of transfer learning model

# 2. TODO: Initialize base model
base_model = ...

# 3. TODO: Freeze layers from the base model
for layer in base_model.layers:
    ...
    
# 4. TODO: Add Fully connected layer to the base model
model = Sequential([...
                    ...
                    ...])

### 3.3 Training Model
First let's compile the model. The optimizer and loss function as already been chosen for you. We are going to use as loss function `sparse_categorical_crossentropy` which is useful when the target contains classes represented as integer values (1 and 0 in our case). The learning rate of Adam was set to 1e-4 since the default value (1e-3) was too big and leading to unstabilities. We'll learning how to better define the learning rate in the next day. 

In [0]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Set the following inputs to the `fit_generator`:

1. The training generator (`train_gen`)
2. The number of batches (`train_gen.n//train_gen.batch_size`)
3. The validation generator (`valid_gen`)
4. The number of validation batches (`valid_gen.n//valid_gen.batch_size`)

In [0]:
# TODO: Set the inputs for training the model using the generators
model.fit_generator(generator=..., 
                    steps_per_epoch=..., 
                    epochs=3,
                    validation_data=..., 
                    validation_steps=...)

## 4. Visualize Predictions
Finally, let's confirm our model is predicting well by using online external sources. Find 5 urls to jpb images of either cats or dogs and fill the `URLS` list. For example:
```
URLS = ['http://example.com/image1.jpg',
        'http://example.com/image2.jpg',
        'http://example.com/image3.jpg',
        'http://example.com/image4.jpg',
        'http://example.com/image5.jpg']
        
```

In [0]:
# TODO: Fill with urls of jpg images 
URLS = ['...',
        '...',
        '...',
        '...',
        '...']

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import requests
from io import BytesIO
from PIL import Image

def predict_from_url(model, train_gen, url, target_shape=(224,224)):
    
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img = img.resize(target_shape)

    # Convert to a Numpy array
    img_np = np.asarray(img)

    # Reshape by adding 1 in the beginning to be compatible as input of the model
    img_np = img_np[None] # https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html#numpy.newaxis

    # Prepare the image for the model
    img_np = preprocess_input(img_np)

    # Decode output of model into classes and probabilities
    result = model.predict(img_np)
 
    class_indices = train_gen.class_indices    
    index_to_class = {v: k for k, v in class_indices.items()}
    
    # Displaying image
    plt.imshow(img)
    plt.title(f'Predicted Class: {index_to_class[result.argmax()]}')
    plt.show()
    
for url in URLS:
    predict_from_url(model, train_gen, url, target_shape=TARGET_SHAPE[:2])  